In [3]:
from manim import *
import numpy as np

class NepaliSolidCorrected(ThreeDScene):
    def construct(self):
        # ==========================================
        # 1. SETUP & CONFIGURATION
        # ==========================================
        
        # --- Dimensions (Real values) ---
        d_val = 10
        r_val = 5
        h_total = 17
        h_hemi = r_val     # 5
        h_cone = h_total - h_hemi # 12
        l_val = np.sqrt(h_cone**2 + r_val**2) # 13
        
        # --- Scaling & Layout ---
        s = 0.25 # Slightly smaller scale to fit height comfortably
        
        # Screen division: Text on LEFT, 3D on RIGHT
        # We place the 3D center at X = 3.5
        RIGHT_ZONE_CENTER = RIGHT * 3.5
        
        # --- Colors ---
        C_CONE = TEAL_E
        C_CONE_STR = TEAL_A
        C_HEMI = BLUE_E
        C_HEMI_STR = BLUE_A
        C_DIM = YELLOW
        
        # --- Fonts ---
        # Note: If 'Noto Sans Devanagari' is not installed, Manim will warn but run.
        font_np = "Noto Sans Devanagari" 

        # ==========================================
        # 2. HELPER FUNCTIONS
        # ==========================================
        def get_dim_line(start, end, text, direction=UP, rotate_axis=RIGHT):
            """Creates a dimension line with text suitable for 3D"""
            line = Line(start, end, color=C_DIM, stroke_width=2)
            line.add_tip(tip_length=0.15, tip_width=0.15)
            line.add_tip(tip_length=0.15, tip_width=0.15, at_start=True)
            
            label = MathTex(text, color=C_DIM, font_size=24)
            midpoint = (start + end) / 2
            
            # Position text offset by direction
            label.move_to(midpoint + direction * 0.4)
            
            # Rotate text to face camera (Billboard effect)
            # Standard setup: Rotate 90 deg around X-axis
            label.rotate(90 * DEGREES, rotate_axis)
            if rotate_axis is OUT:
                 label.rotate(90 * DEGREES, RIGHT)
            
            return VGroup(line, label), label

        # ==========================================
        # 3. OBJECT GENERATION (Centered Logic)
        # ==========================================
        
        # We construct parts relative to an origin (0,0,0) where the 
        # Cone Base meets the Hemisphere Top.
        
        # A. Hemisphere (Bottom)
        # Defined to sit below Z=0
        hemisphere = Surface(
            lambda u, v: np.array([
                (r_val*s) * np.cos(u) * np.sin(v),
                (r_val*s) * np.sin(u) * np.sin(v),
                -(r_val*s) * np.cos(v) # Negative Z for downward curve
            ]),
            u_range=[0, TAU],
            v_range=[0, PI/2],
            resolution=(24, 24)
        )
        hemisphere.set_fill(C_HEMI, opacity=0.8)
        hemisphere.set_style(stroke_width=0.5, stroke_color=C_HEMI_STR)

        # B. Cone (Top)
        # Defined to sit above Z=0
        cone = Cone(
            base_radius=r_val*s,
            height=h_cone*s,
            direction=OUT, # Points up (+Z)
            show_base=True,
            fill_color=C_CONE,
            fill_opacity=0.8
        )
        cone.set_stroke(width=0)
        # Cone defaults to center at origin. We need to shift its BASE to Z=0.
        # The cone spans from -h/2 to +h/2. Base is at -h/2.
        cone.shift(OUT * (h_cone*s / 2)) 

        # C. Rings (The connection point)
        ring = Circle(radius=r_val*s, color=WHITE, stroke_width=2)
        
        # D. GROUPING & CENTERING
        # Create a group to manipulate the whole object
        solid_group = VGroup(hemisphere, cone, ring)
        
        # Calculate Vertical Center shift
        # Top of solid is h_cone * s
        # Bottom of solid is -r_val * s
        # Midpoint Z = (h_cone*s - r_val*s) / 2
        z_center_shift = (h_cone*s - r_val*s) / 2
        
        # Shift everything so the geometric center is at local Z=0
        solid_group.shift(IN * z_center_shift)
        
        # Now move the entire centered group to the Right Zone of the screen
        solid_group.move_to(RIGHT_ZONE_CENTER)
        
        # Store individual references after shifting
        # (Using indices isn't safe if structure changes, so we rely on objects moving with group)
        
        # ==========================================
        # 4. SCENE SETUP
        # ==========================================
        
        self.set_camera_orientation(phi=75 * DEGREES, theta=-10 * DEGREES, zoom=0.9)
        
        # Static UI - Title
        title_en = Text("Combined Solid: Cone + Hemisphere", font_size=28, color=WHITE).to_corner(UL)
        title_np = Text("संयुक्त ठोस वस्तु: सोली र अर्धगोला", font=font_np, font_size=24, color=YELLOW)
        title_np.next_to(title_en, DOWN, aligned_edge=LEFT)
        self.add_fixed_in_frame_mobjects(title_en, title_np)
        
        calc_anchor = title_np.get_bottom() + DOWN * 0.5

        # ==========================================
        # 5. ANIMATION SEQUENCE
        # ==========================================

        # --- Phase 1: Creation (Exploded View) ---
        # We start with them separated vertically
        EXPLODE_DIST = 1.5
        
        # Temporarily shift parts apart
        cone.shift(OUT * EXPLODE_DIST)
        hemisphere.shift(IN * EXPLODE_DIST)
        ring_cone = ring.copy().shift(OUT * EXPLODE_DIST).set_color(TEAL_A)
        ring_hemi = ring.copy().shift(IN * EXPLODE_DIST).set_color(BLUE_A)
        
        self.play(
            GrowFromCenter(hemisphere), Create(ring_hemi),
            GrowFromCenter(cone), Create(ring_cone),
            run_time=1.5
        )
        self.wait(0.5)

        # --- Phase 2: Dimensions ---
        # Radius line on Hemi
        l_r, t_r = get_dim_line(
            ring_hemi.get_center(), ring_hemi.get_right(), "r=5", direction=DOWN, rotate_axis=RIGHT
        )
        
        # Height of Cone (Side marker)
        p_c_top = cone.get_top()
        p_c_base = cone.get_bottom()
        offset = RIGHT * (r_val*s + 0.5)
        l_hc, t_hc = get_dim_line(
            p_c_base + offset, p_c_top + offset, "h_{cone}=?", direction=RIGHT, rotate_axis=RIGHT
        )
        t_hc.rotate(90*DEGREES, OUT) # Adjust reading angle

        self.play(Create(l_r), Create(l_hc))
        
        # --- Phase 3: Calc Part 1 (Find Height) ---
        txt_1 = Text("1. Dimensions", font_size=20, color=BLUE_B).move_to(calc_anchor, aligned_edge=LEFT)
        math_1 = MathTex(r"d=10 \Rightarrow r=5").next_to(txt_1, DOWN, aligned_edge=LEFT)
        math_2 = MathTex(r"Total \ H = 17").next_to(math_1, DOWN, aligned_edge=LEFT)
        math_3 = MathTex(r"h_{cone} = H - r").next_to(math_2, DOWN, aligned_edge=LEFT)
        math_4 = MathTex(r"= 17 - 5 = 12").next_to(math_3, DOWN, aligned_edge=LEFT)
        
        grp_1 = VGroup(txt_1, math_1, math_2, math_3, math_4)
        self.add_fixed_in_frame_mobjects(grp_1)
        self.play(Write(grp_1))
        self.wait(1)
        
        # Update question mark
        t_hc_new = MathTex("h=12", color=YELLOW, font_size=24).move_to(t_hc.get_center())
        t_hc_new.rotate(90*DEGREES, RIGHT).rotate(90*DEGREES, OUT)
        self.play(Transform(t_hc, t_hc_new))
        self.wait(1)

        # --- Phase 4: Assembly (Implosion) ---
        self.play(
            FadeOut(l_r), FadeOut(l_hc), FadeOut(t_hc), FadeOut(ring_cone), FadeOut(ring_hemi)
        )
        
        # Move back to origin intersection
        self.play(
            cone.animate.shift(IN * EXPLODE_DIST),
            hemisphere.animate.shift(OUT * EXPLODE_DIST),
            Create(ring), # The central join ring
            run_time=1.5
        )
        
        # Add Total Height Line
        p_top = cone.get_top()
        p_bot = hemisphere.get_bottom()
        l_H, t_H = get_dim_line(
            p_bot + LEFT*(r_val*s+0.8), p_top + LEFT*(r_val*s+0.8), "H=17", direction=LEFT, rotate_axis=RIGHT
        )
        t_H.rotate(90*DEGREES, OUT)
        self.play(Create(l_H))

        # --- Phase 5: Slant Height ---
        self.play(FadeOut(grp_1))
        
        txt_2 = Text("2. Slant Height (l)", font_size=20, color=TEAL_B).move_to(calc_anchor, aligned_edge=LEFT)
        math_l1 = MathTex(r"l = \sqrt{h^2 + r^2}").next_to(txt_2, DOWN, aligned_edge=LEFT)
        math_l2 = MathTex(r"= \sqrt{12^2 + 5^2}").next_to(math_l1, DOWN, aligned_edge=LEFT)
        math_l3 = MathTex(r"= \sqrt{144+25} = 13").next_to(math_l2, DOWN, aligned_edge=LEFT)
        
        grp_2 = VGroup(txt_2, math_l1, math_l2, math_l3)
        self.add_fixed_in_frame_mobjects(grp_2)
        self.play(Write(grp_2))
        
        # Highlight Slant
        slant_line = Line(cone.get_top(), ring.get_right(), color=YELLOW, stroke_width=4)
        slant_lbl = MathTex("l=13", color=YELLOW, font_size=24).next_to(slant_line, RIGHT)
        slant_lbl.rotate(90*DEGREES, RIGHT)
        
        self.play(Create(slant_line), Write(slant_lbl))
        self.wait(1)
        self.play(FadeOut(slant_line), FadeOut(slant_lbl), FadeOut(l_H))

        # --- Phase 6: TSA & Cost ---
        self.play(FadeOut(grp_2))
        
        txt_3 = Text("3. Area & Cost", font_size=20, color=ORANGE).move_to(calc_anchor, aligned_edge=LEFT)
        math_a1 = MathTex(r"TSA = \pi r(l + 2r)").next_to(txt_3, DOWN, aligned_edge=LEFT)
        math_a2 = MathTex(r"= \pi \times 5(13 + 10)").next_to(math_a1, DOWN, aligned_edge=LEFT)
        math_a3 = MathTex(r"= 115\pi \approx 361.28 cm^2").next_to(math_a2, DOWN, aligned_edge=LEFT)
        math_c1 = MathTex(r"Cost = 361.28 \times 0.40").next_to(math_a3, DOWN, aligned_edge=LEFT)
        math_c2 = MathTex(r"= Rs. 144.51").next_to(math_c1, DOWN, aligned_edge=LEFT)
        
        res_txt = Text("Rs. 150 is Sufficient.", color=GREEN, font_size=24).next_to(math_c2, DOWN, aligned_edge=LEFT)

        grp_3 = VGroup(txt_3, math_a1, math_a2, math_a3, math_c1, math_c2, res_txt)
        self.add_fixed_in_frame_mobjects(grp_3)
        self.play(Write(grp_3))
        
        # Final Rotate
        self.begin_ambient_camera_rotation(rate=0.2)
        self.wait(4)

%manim -ql -v warning NepaliSolidCorrected

Manim Community v0.19.1